In [ ]:
#imports
import openai
import os
client = openai.OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")
print(openai.api_key)

Taking into account token limits, the plan is to have the Cx paste documents into the chat, then the bot will summarize them and highlight differences.

In [ ]:
#user chat loop
messages=[]
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a legal assistant, designed to simplify and highlight differences between legal documents."},
    {"role": "user", "content": "."}
  ]
)

resp = ''
while True:
    resp = input()
    if resp.lower() == 'stop' or resp.lower() == 'exit': break
    
    messages.append({"role": "user", "content": resp})
    print(completion.choices[0].message)
    
with open('outputs.txt', 'a') as out:
    print(list(messages.values))
    #out.writelines(list(messages.values)
    #out.write('\n')
    

In [13]:
import pandas as pd
import json
#LEDGAR dataset load/parse
lines = []
with open(r'LEDGAR_2016-2019_clean.jsonl', encoding='utf8') as line:
    lines = line.read().splitlines()

line_dicts = [json.loads(line) for line in lines]
df_final = pd.DataFrame(line_dicts)

print(df_final)

                                                provision  \
0       Section and Subsection headings in this Amendm...   
1       THIS AMENDMENT AND THE RIGHTS AND OBLIGATIONS ...   
2       This Amendment may be executed in any number o...   
3       The execution and delivery of this Amendment b...   
4       Each of the parties hereto irrevocably waives ...   
...                                                   ...   
846269  Except as expressly provided in this Amendment...   
846270  This Amendment shall be a contract made under ...   
846271  This Amendment may be executed in any number o...   
846272  This Amendment shall be binding upon and shall...   
846273  Any reference to the Credit Agreement or the S...   

                          label  \
0                    [headings]   
1             [applicable laws]   
2                [counterparts]   
3             [binding effects]   
4       [waiver of jury trials]   
...                         ...   
846269         [loan d

In [20]:
print(df_final.source.unique())

['2019/QTR1/000119312519044328/d691151dex101.htm'
 '2019/QTR1/000110465919016051/a19-6809_1ex10d2.htm'
 '2019/QTR1/000110465919016051/a19-6809_1ex10d1.htm' ...
 '2016/QTR4/000155837016010506/iret-20161031ex10171ea91.htm'
 '2016/QTR4/000119312516733212/d260489dex101.htm'
 '2016/QTR4/000119312516760177/d257400dex102.htm']


In [40]:
cont = ''
conts = []
source = ''
count = 0
for line in df_final.itertuples():
    if line.source != source:
        conts.append(cont)
        cont += line.provision + ' '
        source = line.source
    else:
        cont = ''
        cont += line.provision + ' '
conts.append(cont)
conts == conts[1:]
print(conts[:3])

5000
10000
15000
20000
25000
30000
35000
40000
45000
['Notwithstanding anything to the contrary contained herein, if at any time Bank of America assigns all of its Commitment and Loans pursuant to Section\xa010.6(c), Bank of America upon 45 days’ notice to Borrower and the Lenders, may resign as the L/C Issuer. In the event of any such resignation as the L/C Issuer, Borrower shall be entitled to appoint from among the Lenders a successor L/C Issuer hereunder; provided , however , that no failure by Borrower to appoint any such successor shall affect the resignation of Bank of America as the L/C Issuer. If Bank of America resigns as the L/C Issuer, it shall retain all the rights, powers, privileges and duties of the L/C Issuer hereunder with respect to all Letters of Credit issued by it and outstanding, as of the effective date of its resignation as the L/C Issuer, and all L/C Obligations with respect thereto (including the right to require the Lenders to make Base Rate Loans or fund ri

In [27]:
import pandas as pd
df = pd.read_excel("CUAD_v1\label_group_xlsx\Label Report - Anti-assignment, CIC (Group 3).xlsx")
print(df.head())
print(type(df['Filename'][0]))

                                            Filename  \
0  CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
1  EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...   
2  GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...   
3  IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...   
4  DeltathreeInc_19991102_S-1A_EX-10.19_6227850_E...   

                                   Change of Control  \
0                                                NaN   
1                                                NaN   
2  For purposes of the preceding sentence, and wi...   
3                                                NaN   
4  The term of this Agreement shall be effective ...   

                                     Anti-assignment  
0  MA may not assign, sell, lease or otherwise tr...  
1  This Agreement may not be assigned, sold or tr...  
2  Licensee shall not assign or otherwise transfe...  
3  Licensee shall have the right to assign or sub...  
4  This Agreement may not be assigned by DeltaThr..

In [40]:
import openai
import os 
from random import randint
client = openai.OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")

upperlimit = 15
for i in range (0,upperlimit):
    lst = os.listdir("CUAD_v1\\label_group_xlsx")
    rand_category = lst[random.randint(0,len(lst)-1)]
    print(rand_category)
    xlsx = pd.read_excel('CUAD_v1\\label_group_xlsx\\' + rand_category)
    randint1 = randint(0, len(xlsx['Filename']) - 1)
    randint2 = randint(0, len(xlsx['Filename']) - 1)
    cont1 = xlsx['Filename'][randint1]
    cont2 = xlsx['Filename'][randint2]
    cont1 = cont1[:-3].replace("&", "_") + "txt"
    cont2 = cont2[:-3].replace("&", "_") + "txt"
    print(cont1, cont2)
    cont1TXT = ''
    cont2TXT = ''
    with open('CUAD_v1\\full_contract_txt\\' + cont1, "r", encoding='utf8') as out: cont1TXT = out.read()
    with open('CUAD_v1\\full_contract_txt\\' + cont2, "r", encoding='utf8') as out: cont2TXT = out.read()   
    
    
    completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": "You are a legal assistant, designed to simplify and highlight differences between legal documents."},
        {"role": "user", "content": "."}
    ]
    messages.append({"role": "user", "content": "Name the clauses that differ between these two documents: " + cont1TXT
                    "\n" + cont2TXT})
    print(completion.choices[0].message)
    with open('outputs.txt', 'a') as out:
        out.writelines(messages)
        out.write('\n')
)

Label Report - Revenue-Profit Sharing.xlsx
HALITRON,INC_03_01_2005-EX-10.15-SPONSORSHIP AND DEVELOPMENT AGREEMENT.txt SFGFINANCIALCORP_05_12_2009-EX-10.1-SOFTWARE LICENSE AND MAINTENANCE AGREEMENT.txt
Label Report - Governing Law.xlsx
RMRGROUPINC_01_22_2020-EX-99.1-JOINT FILING AGREEMENT.txt PfHospitalityGroupInc_20150923_10-12G_EX-10.1_9266710_EX-10.1_Franchise Agreement3.txt
Label Report - Document Name.xlsx
VIVINT SOLAR, INC. - NON-COMPETITION AGREEMENT.txt PlayboyEnterprisesInc_20090220_10-QA_EX-10.2_4091580_EX-10.2_Content License Agreement_ Marketing Agreement_ Sales-Purchase Agreement2.txt
Label Report - Price Restrictions.xlsx
LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT.txt ELANDIAINTERNATIONALINC_04_25_2007-EX-10.21-Outsourcing Agreement.txt
Label Report - Insurance.xlsx
MEDIWOUNDLTD_01_15_2014-EX-10.6-SUPPLY AGREEMENT.txt ScansourceInc_20190822_10-K_EX-10.38_11793958_EX-10.38_Distributor Agreement1.txt
Label Report - Licenses (Group 4).xlsx
LEGACYTECHNOLOGYHOLDINGS,IN

In [38]:
"SightLife Surgical, Inc. - STRATEGIC SALES _ MARKETING AGREEMENT.txt" == "SightLife Surgical, Inc. - STRATEGIC SALES & MARKETING AGREEMENT.txt'"

False

In [2]:
from docx.enum.text import WD_COLOR_INDEX
import docx

In [ ]:
doc1 = ''
doc2 = ''
comparisonDocument1 = docx.Document()
comparisonDocument2 = docx.Document()
for paragraph in document.split('\n'):
    if clauseNumber in paragraph:
        comparisonDocument1.add_paragraph(paragraph)